In [9]:
from openai import OpenAI
from openai.types.responses import ParsedResponse
import json
import os

In [3]:
prompt = '\nYou are an expert market researcher specializing in Reddit discourse analysis. Your task is to generate 5-8 strategically chosen keywords that will uncover the most relevant discussions about a specific project idea on Reddit.\n\nThese keywords should act as precision search terms to find posts and comments where people are genuinely discussing, experiencing, or seeking solutions related to the project\'s domain.\n\n<instructions>\nGenerate between 5-8 keywords (NEVER MORE THAN 8) that capture multiple dimensions of the project space. Each keyword must be smartly selected and strategically chosen:\n\n**Core Product/Service Keywords:**\n- Direct terms for the main technology, product, or service\n- Industry-specific terminology and jargon\n- Technical specifications and features that matter to users\n\n**Problem-Focused Keywords:**\n- Pain points and frustrations users explicitly mention\n- Common complaints and recurring issues\n- "I wish there was..." or "Why doesn\'t..." type expressions\n- Error messages, bugs, or failure scenarios people discuss\n\n**Solution-Seeking Keywords:**\n- Terms people use when asking for recommendations\n- Comparison language ("X vs Y", "better than", "alternative to")\n- Words indicating research behavior ("looking for", "anyone tried", "reviews")\n\n**Competitor and Market Keywords:**\n- Existing tools, platforms, or solutions in the space\n- Brand names and product names users mention\n- Category names and market segments\n\n**User Journey Keywords:**\n- Terms related to getting started, learning, or adoption challenges\n- Advanced use cases and power user discussions\n- Integration, setup, and configuration topics\n\n**Emotional and Contextual Keywords:**\n- Expressions of frustration, excitement, or urgency\n- Situational contexts where the problem occurs\n- Workflow disruptions or efficiency concerns\n\n**Language Variations:**\n- Include both technical and casual terminology\n- Consider abbreviations, acronyms, and slang\n- Account for different expertise levels (beginner vs expert language)\n- Include typos or common misspellings of technical terms\n\nCRITICAL: You must NEVER generate more than 8 keywords. If fewer high-quality keywords exist, provide only those (minimum 5, maximum 8).\n\nAll keywords must be smartly selected - avoid generic buzzwords, overly broad terms that would return irrelevant results, or keywords that stray from the core project focus. Each keyword should have a clear connection to either the problem the project solves or the solution it provides.\n\nBalance specificity with discoverability - keywords should be narrow enough to find relevant content but broad enough to capture the full conversation landscape.\n</instructions>\n\n<response_format>\nReturn your response as a JSON object following this exact format:\n{\'properties\': {\'keywords\': {\'description\': \'List of keywords\', \'items\': {\'type\': \'string\'}, \'title\': \'Keywords\', \'type\': \'array\'}}, \'required\': [\'keywords\'], \'title\': \'RedditKeywords\', \'type\': \'object\'}\n</response_format> \n\nProject Idea: \nWe want to improve the experience for deaf people during the watching of videos. Deaf people do not hear so they are dependent on visuals of the video and subtitles. The group that is hard of hearing might be fine but I am not sure. I want to figure out what is needed to improve their video experience\n\n'
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name ='gpt-5-nano-2025-08-07'
messages = [{'role': 'user', 'content': '\nYou are an expert market researcher specializing in Reddit discourse analysis. Your task is to generate 5-8 strategically chosen keywords that will uncover the most relevant discussions about a specific project idea on Reddit.\n\nThese keywords should act as precision search terms to find posts and comments where people are genuinely discussing, experiencing, or seeking solutions related to the project\'s domain.\n\n<instructions>\nGenerate between 5-8 keywords (NEVER MORE THAN 8) that capture multiple dimensions of the project space. Each keyword must be smartly selected and strategically chosen:\n\n**Core Product/Service Keywords:**\n- Direct terms for the main technology, product, or service\n- Industry-specific terminology and jargon\n- Technical specifications and features that matter to users\n\n**Problem-Focused Keywords:**\n- Pain points and frustrations users explicitly mention\n- Common complaints and recurring issues\n- "I wish there was..." or "Why doesn\'t..." type expressions\n- Error messages, bugs, or failure scenarios people discuss\n\n**Solution-Seeking Keywords:**\n- Terms people use when asking for recommendations\n- Comparison language ("X vs Y", "better than", "alternative to")\n- Words indicating research behavior ("looking for", "anyone tried", "reviews")\n\n**Competitor and Market Keywords:**\n- Existing tools, platforms, or solutions in the space\n- Brand names and product names users mention\n- Category names and market segments\n\n**User Journey Keywords:**\n- Terms related to getting started, learning, or adoption challenges\n- Advanced use cases and power user discussions\n- Integration, setup, and configuration topics\n\n**Emotional and Contextual Keywords:**\n- Expressions of frustration, excitement, or urgency\n- Situational contexts where the problem occurs\n- Workflow disruptions or efficiency concerns\n\n**Language Variations:**\n- Include both technical and casual terminology\n- Consider abbreviations, acronyms, and slang\n- Account for different expertise levels (beginner vs expert language)\n- Include typos or common misspellings of technical terms\n\nCRITICAL: You must NEVER generate more than 8 keywords. If fewer high-quality keywords exist, provide only those (minimum 5, maximum 8).\n\nAll keywords must be smartly selected - avoid generic buzzwords, overly broad terms that would return irrelevant results, or keywords that stray from the core project focus. Each keyword should have a clear connection to either the problem the project solves or the solution it provides.\n\nBalance specificity with discoverability - keywords should be narrow enough to find relevant content but broad enough to capture the full conversation landscape.\n</instructions>\n\n<response_format>\nReturn your response as a JSON object following this exact format:\n{\'properties\': {\'keywords\': {\'description\': \'List of keywords\', \'items\': {\'type\': \'string\'}, \'title\': \'Keywords\', \'type\': \'array\'}}, \'required\': [\'keywords\'], \'title\': \'RedditKeywords\', \'type\': \'object\'}\n</response_format> \n\nProject Idea: \nWe want to improve the experience for deaf people during the watching of videos. Deaf people do not hear so they are dependent on visuals of the video and subtitles. The group that is hard of hearing might be fine but I am not sure. I want to figure out what is needed to improve their video experience\n\n'} ]
response = llm.chat.completions.create(
            model=model_name, 
            messages=messages)

In [2]:
print(prompt)

NameError: name 'prompt' is not defined

In [34]:
response.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}"

In [37]:
response_dict = {'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}
response_dict

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['captions accuracy',
  'captions sync timing',
  'SDH subtitles',
  'ASL overlay',
  'how to enable captions',
  'YouTube captions quality',
  'caption customization font size contrast']}

In [20]:
response.choices[0].to_dict()

{'finish_reason': 'stop',
 'index': 0,
 'message': {'content': "{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}",
  'refusal': None,
  'role': 'assistant',
  'annotations': []}}

In [22]:
response.to_dict()

{'id': 'chatcmpl-COhSOaSgPtbGbzLJKjFD6OfCxAkW3',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['captions accuracy', 'captions sync timing', 'SDH subtitles', 'ASL overlay', 'how to enable captions', 'YouTube captions quality', 'caption customization font size contrast']}",
    'refusal': None,
    'role': 'assistant',
    'annotations': []}}],
 'created': 1760002956,
 'model': 'gpt-5-nano-2025-08-07',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': None,
 'usage': {'completion_tokens': 2028,
  'prompt_tokens': 665,
  'total_tokens': 2693,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 1920,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_d

In [ ]:
response_1 = llm.chat.completions.create(
            model=model_name, 
            messages=messages,
            reasoning_effort="minimal")
response_1

ChatCompletion(id='chatcmpl-COhpZIHiSmQj8LokQDgqgzLbZbcf9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004393, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=65, prompt_tokens=665, total_tokens=730, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [40]:
response_1.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}"

In [41]:
resonse_1_text = {'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object'}
resonse_1_text

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object'}

In [42]:
response_2 = llm.chat.completions.create(
            model="gpt-5", 
            messages=messages,
            reasoning_effort="minimal")
response_2

ChatCompletion(id='chatcmpl-COhrOHW8qPVa13ZivUHGNUnwzvpZT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004506, model='gpt-5-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(compl

In [ ]:
response_2.choices[0].message.content

"{'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }"

In [45]:
response_2_dict ={'properties': {'keywords': {'description': 'List of keywords', 'items': {'type': 'string'}, 'title': 'Keywords', 'type': 'array'}}, 'required': ['keywords'], 'title': 'RedditKeywords', 'type': 'object', 'keywords': ['open captions vs closed captions', 'subtitles for the deaf and hard of hearing (SDH)', 'auto captions accuracy YouTube Netflix', 'caption timing out of sync complaint', 'visual sound cues (doorbell siren laughter) in captions', 'ASL overlay picture-in-picture video', 'recommendations captioning tools apps', 'accessibility settings video players (HDR contrast font)'] }
response_2_dict

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['open captions vs closed captions',
  'subtitles for the deaf and hard of hearing (SDH)',
  'auto captions accuracy YouTube Netflix',
  'caption timing out of sync complaint',
  'visual sound cues (doorbell siren laughter) in captions',
  'ASL overlay picture-in-picture video',
  'recommendations captioning tools apps',
  'accessibility settings video players (HDR contrast font)']}

In [46]:
response_3 = llm.chat.completions.create(
            model="gpt-5", 
            messages=messages,
            reasoning_effort="high")
response_3

ChatCompletion(id='chatcmpl-COhsuKhABXnsS8HFN0VIkPuv3jxRg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "properties": {\n    "keywords": {\n      "description": "List of keywords",\n      "items": {\n        "type": "string"\n      },\n      "title": "Keywords",\n      "type": "array"\n    }\n  },\n  "required": [\n    "keywords"\n  ],\n  "title": "RedditKeywords",\n  "type": "object",\n  "keywords": [\n    "SDH subtitles",\n    "YouTube auto captions",\n    "Netflix subtitles out of sync",\n    "subtitles too small",\n    "dialogue boost",\n    "Live Caption",\n    "ASL interpreter overlay",\n    "OpenAI Whisper subtitles"\n  ]\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760004600, model='gpt-5-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=5013, prompt_tokens=665, total_toke

In [50]:
json.loads(response_3.choices[0].message.content)

{'properties': {'keywords': {'description': 'List of keywords',
   'items': {'type': 'string'},
   'title': 'Keywords',
   'type': 'array'}},
 'required': ['keywords'],
 'title': 'RedditKeywords',
 'type': 'object',
 'keywords': ['SDH subtitles',
  'YouTube auto captions',
  'Netflix subtitles out of sync',
  'subtitles too small',
  'dialogue boost',
  'Live Caption',
  'ASL interpreter overlay',
  'OpenAI Whisper subtitles']}

## Review

OpenAI is blowing up the tokens with the reasoning. The documentation also changed since last time I used OPENAI, and when the github was created that I found that does pain point research on reddit automatically. Let's try the responses api instead of the chatcompletions api now

In [4]:
from typing import List
from pydantic import BaseModel, Field


class RedditKeywords(BaseModel): 
    keywords: List[str] = Field(description="List of keywords")

In [ ]:


response_responses_1 = llm.responses.parse(model="gpt-5",
                    input=messages,
                    text_format=RedditKeywords)

In [ ]:
response_responses_1.output_parsed

RedditKeywords(keywords=['SDH subtitles', 'YouTube auto captions accuracy', 'subtitle desync streaming', 'no captions on videos', 'Netflix subtitle readability settings', 'open captions vs closed captions', 'HoH friendly video recommendations', 'ASL interpreter overlay'])

In [56]:
response_responses_2 = llm.responses.parse(model="gpt-5-mini",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_2.output_parsed

RedditKeywords(keywords=['closed captions', 'auto-generated captions', 'SDH subtitles', 'caption sync', 'caption accuracy', 'sign language interpreter', 'how to enable captions', 'download subtitles'])

In [23]:
response_responses_3 = llm.responses.parse(model="gpt-5-nano",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_3.output_parsed

RedditKeywords(keywords=['live captions', 'captions accuracy', 'subtitle readability', 'ASL overlay', 'video accessibility', 'scene-by-scene transcripts', 'lip-reading reliance', 'auto-generated vs human captions'])

In [ ]:
response_responses_3.usage.to_dict()

In [5]:
response_responses_4 = llm.responses.parse(model="gpt-4.1",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_4.output_parsed

RedditKeywords(keywords=['captioning accuracy', 'inaccessible videos', 'auto-generated subtitles issues', 'hard of hearing recommendations', 'sign language integration', 'subtitle customization', 'deaf video experience', 'missing audio cues'])

In [27]:
response_responses_4.usage.output_tokens_details.reasoning_tokens

0

In [32]:
f"{1}.json" == "1.json"

True

In [22]:
if not os.path.exists("data"):
    os.mkdir("data")
os.listdir("data")

['search_result.json',
 'cached_comments1.json',
 'post_entity.json',
 'cached_post1.json',
 'comments_result.json',
 'search_result1.json']

In [59]:
response_responses_5 = llm.responses.parse(model="gpt-oss-120b",
                    input=messages,
                    text_format=RedditKeywords)
response_responses_5.output_parsed

RedditKeywords(keywords=['closed captions accuracy', 'auto-generated subtitles errors', 'best captioning tool for deaf', 'YouTube caption sync issue', 'VTT vs SRT for accessibility', 'how to add captions to personal videos', 'deaf-friendly video platforms', 'caption lag frustration'])

In [63]:
response_responses_5.output_parsed.model_dump()

{'keywords': ['closed captions accuracy',
  'auto-generated subtitles errors',
  'best captioning tool for deaf',
  'YouTube caption sync issue',
  'VTT vs SRT for accessibility',
  'how to add captions to personal videos',
  'deaf-friendly video platforms',
  'caption lag frustration']}

In [64]:
response_responses_5.usage

ResponseUsage(input_tokens=665, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=416, output_tokens_details=OutputTokensDetails(reasoning_tokens=256), total_tokens=1081)